In [1]:
pwd

'/Users/peterhaglich/Dropbox/Work/IARPA/Mercury/peterhaglich/mercury-challenge-team/scoring_code/src/test/scoring_test/GSR'

In [2]:
%matplotlib inline

In [3]:
import os
import json
import sys
import re

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use("fivethirtyeight")

from dateutil.parser import parse
import datetime

from geopy.distance import vincenty

In [4]:
MERC_HOME = os.path.join("..", "..", "..", "..", "..", "..", "mercury")
MERC_DATA_PATH = os.path.join(MERC_HOME, "data")
MERC_GSR_PATH = os.path.join(MERC_DATA_PATH, "gsr")
MERC_MA_PATH = os.path.join(MERC_GSR_PATH, "mansa_gsr")
MCT_HOME = os.path.join("..", "..", "..", "..", "..")
MCT_DATA_PATH = os.path.join(MCT_HOME, "data")
MCT_GSR_PATH = os.path.join(MCT_DATA_PATH, "gsr")
MCT_MA_PATH = os.path.join(MCT_GSR_PATH, "ma_gsr")
MCT_SRC_PATH = os.path.join(MCT_HOME, "scoring_code", "src", "main", "python", "mercury", "scoring")

In [5]:
ma_gsr = []
ma_files = [x for x in os.listdir(MERC_MA_PATH) if "Dup" not in x
            if x.startswith("MA")
            if x.endswith("json")]
ma_files

['MANSA_November_2017.json',
 'MANSA_March_2017.json',
 'MANSA_July_2017.json',
 'MANSA_July_2016.json',
 'MANSA_June_2015.json',
 'MANSA_February_2018.json',
 'MANSA_March_2016.json',
 'MANSA_April_2018.json',
 'MANSA_September_2015.json',
 'MANSA_November_2016.json',
 'MANSA_May_2015.json',
 'MANSA_October_2015.json',
 'MANSA_December_2016.json',
 'MANSA_August_2016.json',
 'MANSA_January_2018.json',
 'MANSA_August_2017.json',
 'MANSA_December_2017.json',
 'MANSA_October_2017.json',
 'MANSA_January_2016.json',
 'MANSA_January_2017.json',
 'MANSA_October_2016.json',
 'MANSA_December_2015.json',
 'MANSA_August_2015.json',
 'MANSA_February_2017.json',
 'MANSA_September_2016.json',
 'MANSA_May_2016.json',
 'MANSA_November_2015.json',
 'MANSA_June_2016.json',
 'MANSA_July_2015.json',
 'MANSA_April_2017.json',
 'MANSA_April_2016.json',
 'MANSA_June_2017.json',
 'MANSA_March_2018.json',
 'MANSA_May_2017.json',
 'MANSA_September_2017.json',
 'MANSA_February_2016.json']

In [6]:
for file_ in ma_files:
    try:
        path_ = os.path.join(MERC_MA_PATH, file_)
        with open(path_, "r", encoding="utf8") as f:
            gsr_ = json.load(f)
            ma_gsr += gsr_
    except json.JSONDecodeError:
        print(file_)

In [7]:
ma_gsr_df = pd.DataFrame(ma_gsr)
ma_gsr_df = ma_gsr_df[ma_gsr_df.Event_Type == "Military Action"]

In [8]:
# Replace duplicated event IDs with the latest version.
ma_gsr_df["Revision_DTG"] = ma_gsr_df.Revision_Date.apply(parse)
for ed, group in ma_gsr_df.groupby("Event_ID"):
    newest_index = group.Revision_DTG.argmax()
    newest_rev_date = group.Revision_Date[newest_index]
    ma_gsr_df.set_value(group.index[0], "Revision_Date", newest_rev_date)
ma_gsr_df.drop("Revision_DTG", axis=1, inplace=True)
ma_gsr_df.drop_duplicates(subset="Event_ID", keep="last", inplace=True)

In [9]:
ma_gsr_df.Country.value_counts()

Syria           64066
Iraq            19562
Egypt            1972
Lebanon          1321
Yemen             443
Saudi Arabia      175
Jordan             28
Bahrain            27
Turkey              2
Qatar               2
Libya               1
Palestine           1
Name: Country, dtype: int64

Remove Target fields from the Challenge GSR

In [12]:
drop_fields = [x for x in ma_gsr_df.columns if "Target" in x]
for f in drop_fields:
    ma_gsr_df.drop(f, axis=1, inplace=True)

In [13]:
ma_gsr_df.iloc[-2]

Actor                                             Unspecified
Actor_Status                                              NaN
Approximate_Location                                    False
Casualties                                       Not Reported
City                                                 Munbaţiḩ
Country                                                 Syria
Earliest_Reported_Date                             2016-02-29
Encoding_Comment                                         None
Event_Date                                         2016-02-29
Event_ID                                              MN34237
Event_Subtype                                  Armed Conflict
Event_Type                                    Military Action
First_Reported_Link        http://www.syriahr.com/en/?p=44624
GSS_Link                   http://www.syriahr.com/en/?p=44624
Latitude                                              36.7008
Longitude                                             38.9414
News_Sou

## Export Deduplicated Monthly GSR

In [14]:
ma_gsr_df["YEARMON"] = ma_gsr_df.Event_Date.apply(lambda x: parse(x).strftime("%B_%Y"))
for yearmon in ma_gsr_df.YEARMON.unique():
    df_ = ma_gsr_df[ma_gsr_df.YEARMON == yearmon]
    df_.drop("YEARMON", axis=1, inplace=True)
    out_filename = "MA_{}.json".format(yearmon)
    out_path = os.path.join(MCT_MA_PATH, out_filename)
    out_json = df_.to_json(force_ascii=False, orient="records", path_or_buf=out_path)

/anaconda3/envs/mercury/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
